# 첫 번재 심층 신경망

## 라이브러리 임포트

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Flatten, Dense, Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

from keras.datasets import cifar10
# 6만개의 이미지로 구성되어 있고 각 이미지는 10개의 클래스 중 하나로 분류되어 있음

## 데이터 적재

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# numpy 배열
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

170498071/170498071 [==============================] - 10s 0us/step
(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [3]:
NUM_CLASSES = 10

In [4]:
# 기본적으로 이미지 데이터는 픽셀 채널마다 0~255 사이의 정수로 구성됨
# 신경망은 입력이 -1~1 사이 범위에 놓여 있을 때 가장 잘 동작하기 때문에 데이터를 255로 나눔
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

# 이미지의 정수 레이블을 원핫인코딩 벡터로 바꿈
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [5]:
# 첫번째 차원 : 데이터셋에 있는 이미지의 인덱스
# 두번째 차원 : 이미지의 높이
# 세번째 차원 : 이미지의 너비
# 네번째 차원 : 컬러 채널(RGB이미지이므로 빨강, 초록, 파랑)  => 4차원 텐서

# 인덱스 54 이미지에서 (12,13) 위치에 해당하는 픽셀의 초록 채널(1) 값을 의미함
X_train[54, 12, 13, 1]

0.36862746

## 모델 만들기

In [ ]:
# Sequential 모델을 사용한 네트워크
# 일렬로 층을 쌓은 네트워크를 빠르게 만들 때 사용하기 좋음

model = Sequential([
    Dense(200, activation='relu', input_shape=(32,32,3)),
    Flatten(),
    Dense(150, activation='relu'),
    Dense(10, activation='softmax')
])

In [ ]:
# 함수형 API(Model)를 사용한 네트워크
# 한 층이 여러개의 별도의 층으로 전달되거나 한 층이 여러 층으로부터 입력 받을때 사용하기 좋음. 유연성

# Input층 : 네트워크의 시작점. 입력 데이터 크기를 튜플로 알려주어야함. batch_size 지정하지 않음.
    # Input 층에 임의의 이미지 개수를 전달할 수 있기 때문에 batch_size는 필요하지 않음
input_layer = Input(shape=(32,32,3))

# Flatten층 : 입력을 하나의 벡터로 펼침 >> 결과 벡터의 길이 : 32 x 32 x 3 = 3072
    # 뒤따르는 Dense 층이 다차원 배열이 아니라 평평한 입력을 기대하기 때문
x = Flatten()(input_layer)

# Dense 층 : 이전 층과 완전하게 연결되는 유닛을 가지고 있음. 연결마다 하나의(양수 또는 음수인) 가중치가 동반됨
    # 유닛의 출력은 이전 층으로부터 받은 입려과 가중치를 곱하여 더한 것
    # 비선형 활성화 함수를 통과하고 다음 층으로 전달됨 
x = Dense(units=200, activation='relu')(x)
x = Dense(units=150, activation='relu')(x)

output_layer = Dense(units=10, activation='softmax')(x)

model = Model(input_layer, output_layer)